In [1]:
from util import *
from glob import glob

In [2]:
df = load_AOIs()
df

,Taranaki,AOI,SSP 4.5 (p50),SSP 4.5 (p83),SSP 8.5 (p50),SSP 8.5 (p83),Rate SSP 4.5 (p50),Rate SSP 4.5 (p83),Rate SSP 8.5 (p50),Rate SSP 8.5 (p83),match,match_score
7,NORTH,TongaporutuRiver,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,TongapurutuRiverCliffs,93.750000
11,SOUTH,HangatahuaRiver_South,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,HangatahuRiver_South,97.435897
21,SOUTH,Rahotu,0.58,0.78,0.84,1.10,0.0058,0.0078,0.0084,0.0110,Rahotu,100.000000
20,SOUTH,Pihama,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Pihama,100.000000
19,SOUTH,OpunakeBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,OpunakeBeachCliffs,100.000000
18,SOUTH,OhaweBeach,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,OhaweBeach,100.000000
17,SOUTH,Oeo,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,Oeo,100.000000
16,SOUTH,Manutahi,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,Manutahi,100.000000
15,SOUTH,ManaBay,0.57,0.78,0.83,1.10,0.0057,0.0078,0.0083,0.0110,ManaBayCliffs,100.000000
14,SOUTH,KaupokonuiBeach,0.57,0.78,0.84,1.10,0.0057,0.0078,0.0084,0.0110,KaupokonuiBeach,100.000000


In [3]:
site = df.match.sample(1).iloc[0]
site

'OnaeroCliff'

In [4]:
gdf = gpd.read_file(f"Shapefiles/{site}_intersects.shp")
gdf.crs = 2193
gdf = enrich_df(gdf)
transect_metadata = get_transect_metadata(f"Shapefiles/{site}_TransectLines.shp")
if site == "ManaBayCliffs":
  print("Flipping")
  for k, v in transect_metadata.items():
    transect_metadata[k]["Azimuth"] = v["Azimuth"] + 180

In [5]:
linear_models = fit(gdf, transect_metadata)
linear_models.loc[linear_models.slope > 0, "slope"] = pd.NA
rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
linear_models.slope = rolled_slopes.slope
linear_models.dropna(inplace=True)
linear_models.index = linear_models.TransectID
linear_models

,TransectID,slope,intercept,group
TransectID,,,,
2,2,0.000000,30.615606,0
3,3,-0.034855,37.162840,0
4,4,-0.055821,38.989387,0
5,5,-0.069328,39.071136,0
6,6,-0.082627,40.521100,0
...,...,...,...,...
751,751,-0.115850,48.197881,19
752,752,-0.125236,52.361688,19
753,753,-0.143928,65.355207,19


In [6]:
# Find the closest data point to 2000
gdf["years_since_2000"] = (gdf.Year - 2000).abs()
closest_to_2000 = gdf.loc[gdf.groupby("TransectID").years_since_2000.idxmin()]
closest_to_2000.set_index("TransectID", inplace=True)
closest_to_2000

,TransOrder,BaselineID,ShorelineI,Distance,IntersectX,IntersectY,Uncertaint,geometry,Date,Year,YearsSinceBase,YearsUntilFuture,years_since_2000
TransectID,,,,,,,,,,,,,
2,2,1,21/11/2017,30.615606,1.713854e+06,5.684033e+06,3.0,POINT (1713853.826 5684033.243),2017-11-21,2017,77.889117,82.110883,17
3,3,1,21/11/2017,31.733252,1.713859e+06,5.684025e+06,3.0,POINT (1713859.300 5684025.293),2017-11-21,2017,77.889117,82.110883,17
4,4,1,21/11/2017,31.375433,1.713864e+06,5.684016e+06,3.0,POINT (1713863.633 5684016.407),2017-11-21,2017,77.889117,82.110883,17
5,5,1,21/11/2017,30.467812,1.713868e+06,5.684007e+06,3.0,POINT (1713867.554 5684007.155),2017-11-21,2017,77.889117,82.110883,17
6,6,1,21/11/2017,29.673871,1.713872e+06,5.683998e+06,3.0,POINT (1713871.574 5683997.959),2017-11-21,2017,77.889117,82.110883,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,751,1,21/11/2017,38.649898,1.720259e+06,5.683558e+06,3.0,POINT (1720259.448 5683557.874),2017-11-21,2017,77.889117,82.110883,17
752,752,1,21/11/2017,41.576230,1.720267e+06,5.683552e+06,3.0,POINT (1720266.892 5683551.713),2017-11-21,2017,77.889117,82.110883,17
753,753,1,21/11/2017,46.567269,1.720273e+06,5.683546e+06,3.0,POINT (1720272.936 5683545.914),2017-11-21,2017,77.889117,82.110883,17


In [7]:
closest_to_2000.Year.value_counts()

2011    354
2017    117
1979     64
2022     47
1970      4
1950      4
Name: Year, dtype: int64

In [8]:
# Predict to the closest year to 2000
linear_models["FUTURE_YEAR"] = closest_to_2000.Year
linear_models

,TransectID,slope,intercept,group,FUTURE_YEAR
TransectID,,,,,
2,2,0.000000,30.615606,0,2017
3,3,-0.034855,37.162840,0,2017
4,4,-0.055821,38.989387,0,2017
5,5,-0.069328,39.071136,0,2017
6,6,-0.082627,40.521100,0,2017
...,...,...,...,...,...
751,751,-0.115850,48.197881,19,2017
752,752,-0.125236,52.361688,19,2017
753,753,-0.143928,65.355207,19,2017


In [9]:
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,sqrt_model_point,BH_model_point,Sunamura_model_point
0,2.0,1,0.0,2017,POINT (1714244.3136806744 5684345.523506876),POINT (1713853.8258120483 5684033.242824707),POINT (1713853.8258120483 5684033.242824707),POINT (1713840.4719375696 5684022.563473652),POINT (1713849.7016860647 5684029.944681592)
1,3.0,1,0.0,2017,POINT (1714247.708260327 5684340.157088014),POINT (1713861.4330582872 5684027.022574091),POINT (1713858.8560754084 5684024.933539443),POINT (1713848.1503051233 5684016.254892311),POINT (1713857.3326384847 5684023.698563244)
2,4.0,1,0.0,2017,POINT (1714250.110816491 5684333.637134484),POINT (1713866.1961886217 5684018.510470374),POINT (1713862.0895654545 5684015.139652058),POINT (1713852.9794589002 5684007.661850276),POINT (1713862.1171928116 5684015.162329279)
3,5.0,1,0.0,2017,POINT (1714252.2534011616 5684326.538637022),POINT (1713870.0657584276 5684009.240342748),POINT (1713864.9889074087 5684005.025459826),POINT (1713856.9098246489 5683998.31807609),POINT (1713866.0061941312 5684005.870027535)
4,6.0,1,0.0,2017,POINT (1714254.5960650032 5684319.352728396),POINT (1713875.0095290607 5684000.841806086),POINT (1713868.9852132446 5683995.786804907),POINT (1713861.9109642133 5683989.850805154),POINT (1713870.968322518 5683997.4508311665)
...,...,...,...,...,...,...,...,...,...
566,751.0,1,19.0,2017,POINT (1720544.0803993323 5683963.465754068),POINT (1720259.8087861126 5683558.387321446),POINT (1720253.4749292978 5683549.361767463),POINT (1720249.9865620562 5683544.390949876),POINT (1720256.7796330887 5683554.0708702)
567,752.0,1,19.0,2017,POINT (1720553.9700926228 5683953.736093098),POINT (1720267.5558842479 5683552.642574393),POINT (1720260.6290266516 5683542.942225718),POINT (1720257.6191790455 5683538.727244995),POINT (1720264.4917763423 5683548.351607725)
568,753.0,1,19.0,2017,POINT (1720564.6327758385 5683946.076920035),POINT (1720277.4749535206 5683552.1408485565),POINT (1720269.405701503 5683541.071084153),POINT (1720267.4027188406 5683538.323302063),POINT (1720274.3697615797 5683547.881005992)
569,754.0,1,19.0,2017,POINT (1720575.9976992952 5683938.304334817),POINT (1720290.39296461 5683555.275722912),POINT (1720280.201504888 5683541.607809896),POINT (1720280.1718023038 5683541.567975334),POINT (1720287.2432129926 5683551.051545776)


In [10]:
results.index = results.TransectID.astype(int)
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,sqrt_model_point,BH_model_point,Sunamura_model_point
TransectID,,,,,,,,,
2,2.0,1,0.0,2017,POINT (1714244.3136806744 5684345.523506876),POINT (1713853.8258120483 5684033.242824707),POINT (1713853.8258120483 5684033.242824707),POINT (1713840.4719375696 5684022.563473652),POINT (1713849.7016860647 5684029.944681592)
3,3.0,1,0.0,2017,POINT (1714247.708260327 5684340.157088014),POINT (1713861.4330582872 5684027.022574091),POINT (1713858.8560754084 5684024.933539443),POINT (1713848.1503051233 5684016.254892311),POINT (1713857.3326384847 5684023.698563244)
4,4.0,1,0.0,2017,POINT (1714250.110816491 5684333.637134484),POINT (1713866.1961886217 5684018.510470374),POINT (1713862.0895654545 5684015.139652058),POINT (1713852.9794589002 5684007.661850276),POINT (1713862.1171928116 5684015.162329279)
5,5.0,1,0.0,2017,POINT (1714252.2534011616 5684326.538637022),POINT (1713870.0657584276 5684009.240342748),POINT (1713864.9889074087 5684005.025459826),POINT (1713856.9098246489 5683998.31807609),POINT (1713866.0061941312 5684005.870027535)
6,6.0,1,0.0,2017,POINT (1714254.5960650032 5684319.352728396),POINT (1713875.0095290607 5684000.841806086),POINT (1713868.9852132446 5683995.786804907),POINT (1713861.9109642133 5683989.850805154),POINT (1713870.968322518 5683997.4508311665)
...,...,...,...,...,...,...,...,...,...
751,751.0,1,19.0,2017,POINT (1720544.0803993323 5683963.465754068),POINT (1720259.8087861126 5683558.387321446),POINT (1720253.4749292978 5683549.361767463),POINT (1720249.9865620562 5683544.390949876),POINT (1720256.7796330887 5683554.0708702)
752,752.0,1,19.0,2017,POINT (1720553.9700926228 5683953.736093098),POINT (1720267.5558842479 5683552.642574393),POINT (1720260.6290266516 5683542.942225718),POINT (1720257.6191790455 5683538.727244995),POINT (1720264.4917763423 5683548.351607725)
753,753.0,1,19.0,2017,POINT (1720564.6327758385 5683946.076920035),POINT (1720277.4749535206 5683552.1408485565),POINT (1720269.405701503 5683541.071084153),POINT (1720267.4027188406 5683538.323302063),POINT (1720274.3697615797 5683547.881005992)


In [11]:
results["True"] = closest_to_2000.geometry
results

,TransectID,BaselineID,group,Year,ocean_point,linear_model_point,sqrt_model_point,BH_model_point,Sunamura_model_point,True
TransectID,,,,,,,,,,
2,2.0,1,0.0,2017,POINT (1714244.3136806744 5684345.523506876),POINT (1713853.8258120483 5684033.242824707),POINT (1713853.8258120483 5684033.242824707),POINT (1713840.4719375696 5684022.563473652),POINT (1713849.7016860647 5684029.944681592),POINT (1713853.826 5684033.243)
3,3.0,1,0.0,2017,POINT (1714247.708260327 5684340.157088014),POINT (1713861.4330582872 5684027.022574091),POINT (1713858.8560754084 5684024.933539443),POINT (1713848.1503051233 5684016.254892311),POINT (1713857.3326384847 5684023.698563244),POINT (1713859.300 5684025.293)
4,4.0,1,0.0,2017,POINT (1714250.110816491 5684333.637134484),POINT (1713866.1961886217 5684018.510470374),POINT (1713862.0895654545 5684015.139652058),POINT (1713852.9794589002 5684007.661850276),POINT (1713862.1171928116 5684015.162329279),POINT (1713863.633 5684016.407)
5,5.0,1,0.0,2017,POINT (1714252.2534011616 5684326.538637022),POINT (1713870.0657584276 5684009.240342748),POINT (1713864.9889074087 5684005.025459826),POINT (1713856.9098246489 5683998.31807609),POINT (1713866.0061941312 5684005.870027535),POINT (1713867.554 5684007.155)
6,6.0,1,0.0,2017,POINT (1714254.5960650032 5684319.352728396),POINT (1713875.0095290607 5684000.841806086),POINT (1713868.9852132446 5683995.786804907),POINT (1713861.9109642133 5683989.850805154),POINT (1713870.968322518 5683997.4508311665),POINT (1713871.574 5683997.959)
...,...,...,...,...,...,...,...,...,...,...
751,751.0,1,19.0,2017,POINT (1720544.0803993323 5683963.465754068),POINT (1720259.8087861126 5683558.387321446),POINT (1720253.4749292978 5683549.361767463),POINT (1720249.9865620562 5683544.390949876),POINT (1720256.7796330887 5683554.0708702),POINT (1720259.448 5683557.874)
752,752.0,1,19.0,2017,POINT (1720553.9700926228 5683953.736093098),POINT (1720267.5558842479 5683552.642574393),POINT (1720260.6290266516 5683542.942225718),POINT (1720257.6191790455 5683538.727244995),POINT (1720264.4917763423 5683548.351607725),POINT (1720266.892 5683551.713)
753,753.0,1,19.0,2017,POINT (1720564.6327758385 5683946.076920035),POINT (1720277.4749535206 5683552.1408485565),POINT (1720269.405701503 5683541.071084153),POINT (1720267.4027188406 5683538.323302063),POINT (1720274.3697615797 5683547.881005992),POINT (1720272.936 5683545.914)


In [12]:
m = gpd.GeoSeries(results.linear_model_point, crs=2193).explore(color="blue", tiles="Esri.WorldImagery")
closest_to_2000.explore(color="green", m=m)

In [13]:
for model in SUPPORTED_MODELS:
  print(model)
  print(gpd.GeoSeries(results[f"{model}_model_point"], crs=2193).distance(results["True"]).describe())

linear
count    571.000000
mean       7.060036
std        8.185067
min        0.000000
25%        1.777410
50%        4.795599
75%        9.535219
max       64.656363
dtype: float64
sqrt
count    571.000000
mean      19.329757
std       17.625742
min        0.000000
25%        4.465307
50%       13.700844
75%       31.445184
max       75.383025
dtype: float64
BH
count    571.000000
mean      16.514485
std        9.159825
min        0.003291
25%       10.377830
50%       15.956706
75%       20.660365
max       76.349629
dtype: float64
Sunamura
count    571.000000
mean       8.645187
std        8.013691
min        0.005699
25%        3.716650
50%        6.102188
75%       10.750266
max       64.520500
dtype: float64
